In [ ]:
# load data

from pathlib import Path

data_dir = Path('./data/wikiart/')


# count images
cnt = 0
for dir in data_dir.iterdir():
    if dir.is_dir():
        for file in dir.iterdir():
            if str(file).endswith('jpg'):
                cnt+=1
cnt

81446

In [11]:
import json
import re
import shutil
from PIL import Image
from tqdm import tqdm
from pathlib import Path

save_dir='./data/wikiart-laion/'
shutil.rmtree(save_dir)
save_dir=Path(save_dir).absolute()
save_dir.mkdir(parents=True,exist_ok=True)
#remove save_dir and content if it exists

iter=0
for style_dir in tqdm(data_dir.iterdir()):
    if style_dir.is_dir():
        for img_file in style_dir.glob('*.jpg'):

            #IMAGES
            img = Image.open(img_file)
            width,height = img.size
            if height < 64 or width < 64:
                continue
            elif height > 512 and width > 512:
                #change largest dimension to 512 and keep aspect ratio
                if height > width:
                    resized_img = img.resize((512, int(512*height/width)), Image.ANTIALIAS)
                    resized_img.save(save_dir/str(str(iter)+'.jpg'))

                else:
                    resized_img = img.resize((int(512*width/height), 512), Image.ANTIALIAS)
                    resized_img.save(save_dir/str(str(iter)+'.jpg'))

            elif height > 512:
                #resize to 512 and keep aspect ratio
                resized_img = img.resize((512, int(512*height/width)), Image.ANTIALIAS)
                #save image as resized
                resized_img.save(save_dir/str(str(iter)+'.jpg'))
            elif width > 512:
                #resize to 512 and keep aspect ratio
                resized_img = img.resize((int(512*width/height), 512), Image.ANTIALIAS)
                #save image as resized
                resized_img.save(save_dir/str(str(iter)+'.jpg'))
            else:    
                img.save(save_dir/str(str(iter)+'.jpg'))
            
            #CREATE CAPTION
            style=str(style_dir.name)
            style=style.replace('_', ' ')# fix underscore in style
            
            # fix capitals in style
            style=style.lower()
            style=style[0].upper()+style[1:]

            # fix capitals in artist name
            artist_name=str(img_file.name).split('_')[0]
            artist_name=artist_name.replace('-', ' ')
            artist_name=artist_name.title()
            
            # fix artwork name
            artwork_name=str(img_file.name).split('_')[1]
            artwork_name=artwork_name.replace('.jpg', '')
            artwork_name=artwork_name.replace('-', ' ')
            artwork_name=re.sub('\(\d\)', '', artwork_name)
            artwork_name=artwork_name.title()
            artwork_name=re.sub('\d+', '', artwork_name)
            artwork_name=artwork_name.strip()
            
            # TODO Parsing of year form filename not working correctly
            # if '-' in artwork_name:
            #     artwork_name=artwork_name.split('-')[0]

            # #find year in artwork name
            # def find_year(s):
            #     if re.match('\d{4}',s):
            #         return int(re.match('\d{4}',s)[0])
            # year=find_year(artwork_name)
            # #years=[_ for _ in years if _]
            # if year:
            #     caption=f"{style} artwork by artist {artist_name}. The artwork is called {artwork_name} and was created in {year}."
            # else:
            #     caption=f"{style} artwork by artist {artist_name}. The artwork is called {artwork_name}."
            caption=f"{style} artwork by artist {artist_name}. The artwork is called {artwork_name}."
            #save caption
            with open(save_dir/str(str(iter)+'.txt'),'w+') as f:
                f.write(caption)
            
            #CREATE METADATA
            if resized_img:
                width,height = resized_img.size
            
            meta_data={'url':'https://github.com/cs-chan/ArtGAN/tree/master/WikiArt%20Dataset','width':width,'height':height}
            #save metadata
            with open(save_dir/str(str(iter)+'.json'),'w+') as f:
                f.write(json.dumps(meta_data))
            iter+=1



28it [11:13, 22.90s/it]